# **💁🏻🗨️💁🏻‍♂️안개 예측 EDA code**
> **안개량 예측** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최대 10명이 참여할 수 있는 기상청 주관 날씨 빅데이터 경진대회 입니다.     
> 주어진 데이터를 활용하여 안개 상태의 구간을 예측할 수 있는 모델을 만드는 것이 목표입니다!

# Contents  
  
- 필요한 라이브러리 설치  
- 데이터 불러오기  
- 사용할 변수 선택하기
- 모델링
- 추론
- 결과 저장하기
- 결과 그 이후

### 1. 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [14]:
# basic
import os, random
import pandas as pd
import numpy as np
import torch

# model
from catboost import CatBoostRegressor, CatBoostClassifier, Pool

# eval metric
from sklearn.metrics import confusion_matrix

# k-fold by timeseries split
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold

# graph
import shap
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [15]:
# random seed 고정하기
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

### 2. 데이터 불러오기
- 제공된 데이터를 불러옵니다.

> - year : 년도
> - month : 월
> - day : 일
> - hour : 시간
> - minute : 분(10분 단위)
> - stn_id : 지점 번호
> - ws10_deg : 10분 평균 풍향, deg
> - ws10_ms : 10분 평균 풍속, m/s
> - ta : 1분 평균 기온 10분 주기, 섭씨
> - re : 강수 유무 0:무강수, 1:강수
> - hm : 1분 평균 상대 습도 10분 주기, %
> - sun10 : 1분 일사량 10분 단위 합계, MJ
> - ts : 1분 평균 지면온도 10분 주기, 섭씨

- test 없는 데이터 값
> - vis1 : 1분 평균 시정 10분 주기, m
> - class : 시정 구간

시정 구간은 다음과 같다.
- 0초과 200미만 : 1
- 200이상 500미만 : 2
- 500이상 1000미만 : 3
- 1000이상 : 4
- 4번은 맞춰도 스코어가 증가하진 않지만 틀리면 감점

In [16]:
# load makes data
train = pd.read_csv('../data/train_preprocessed_data.csv')
test = pd.read_csv('../data/test_preprocessed_data.csv')

### 3. 사용할 변수 선택하기

데이터에서 필요한 칼럼을 선택하여 적용하기로 한다.

In [17]:
# onehotencoder in ground
# train['A'] = np.where(train['ground'] == 'A', 1, 0)
# train['B'] = np.where(train['ground'] == 'B', 1, 0)
# train['C'] = np.where(train['ground'] == 'C', 1, 0)
# train['D'] = np.where(train['ground'] == 'D', 1, 0)

# test['A'] = np.where(test['ground'] == 'A', 1, 0)
# test['B'] = np.where(test['ground'] == 'B', 1, 0)
# test['C'] = np.where(test['ground'] == 'C', 1, 0)
# test['D'] = np.where(test['ground'] == 'D', 1, 0)

#### 단순히 I, J년 train, K년 valid로 활용할 때 사용할 코드

In [18]:
# label 선택하기
use_label_x = ['hm', 're', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'ground', # 'A', 'B', 'C', 'D',
       'dew_point', 'sin_time', 'cos_time', 'sin_month', 'cos_month', 'diff_air-dew', 'diff_ts-dew', 'fog_risk',
       'make_copyfog', 'make_irufog', 'make_mountfog', 'make_gimfog', 'retain_fog', 'upclass_fog'
       ]

use_label_y = ['class']

In [19]:
# train_x, train_y, test_x 만들자
# train: I,J년도, valid: K년도, test: L년도
# train_x = train.loc[(train['year'].isin(['I', 'J'])), use_label_x]
# train_y = train.loc[(train['year'].isin(['I', 'J'])), use_label_y]

# valid_x = train.loc[train['year'].isin(['K']), use_label_x]
# valid_y = train.loc[train['year'].isin(['K']), use_label_y]

# test_x = test[use_label_x]

#### StratifiedKFoldsplit

In [20]:
# 지정 label
use_label = [
    'hm', 're', 'sun10', 'ta', 'ts', 'ws10_deg', 'ws10_ms', 'ground', # 'A', 'B', 'C', 'D',
    'dew_point', 'sin_time', 'cos_time', 'sin_month', 'cos_month', 'diff_air-dew', 'diff_ts-dew', 'fog_risk',
    # 'make_copyfog', 'make_irufog', 'make_mountfog', 'make_gimfog', 'retain_fog', 'upclass_fog',
    'class'
]

cat_features = [
    're', 'ground'#, # 'A', 'B', 'C', 'D',
    # 'make_copyfog', 'make_irufog', 'make_mountfog', 'make_gimfog', 'retain_fog', 'upclass_fog'
]

In [21]:
# 사용할 변수만 넣어주기
use_train = train[use_label]
use_test = test[use_label]

In [22]:
# parameter 지정
# gap: train 이후 몇개를 사용하지 않을것인지 정하기 위한 파라미터
tscv = StratifiedKFold(n_splits = 6, random_state=42, shuffle = True)

In [23]:
# train_x, train_y, test_x 만들자
train_x = use_train[use_train.columns.difference(['class'])]
train_y = use_train['class']

test_x = test[use_train.columns.difference(['class'])]

### 4. 모델링
Catboost는 시계열 데이터 또는 범주형 데이터에 좋은 성능을 가져오는 모델로 알려져 있기 때문에 적용하도록 한다.

timeseries split을 활용해 적합을 진행해보도록 한다.

In [24]:
# custom metric
class CSIMetric(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight):
        best_class = np.zeros(len(approxes[0]))
        
        for i in range(len(approxes[0])):
            approx_i = [approxes[j][i] for j in range(len(approxes))]
            best_class[i] = np.argmax(np.array(approx_i))
        
        accuracy_sum = 0
        weight_sum = 0 

        for i in range(len(target)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            if best_class[i] != '4' or best_class[i] != 4:
                accuracy_sum += w * (best_class[i] == target[i])

        return accuracy_sum, weight_sum

In [25]:
# time series cv fitting models

# 해당 스코어와 모델 저장 리스트
scores = []
models = []

# split마다 모델 적합하기
for train_idx, valid_idx in tscv.split(use_train[use_train.columns.difference(['class'])], use_train['class']):
    print("="*50)
    
    # 가중치 만들기
    class_weights = [30, 30, 30, 1]


    # best parameter - 웬만하면 이 파라미터로 계속 진행할 예정
    cbrm_trial_params = {'objective': 'MultiClass',
                        'depth': 12, 
                        'boosting_type': 'Plain', 
                        'bootstrap_type': 'Bernoulli', 
                        'iterations': 2178, 
                        'learning_rate': 0.03978756063905378, 
                        'reg_lambda': 95.06719663989301, 
                        'subsample': 0.980507422817941, 
                        'random_strength': 42.091243716971206, 
                        'min_data_in_leaf': 61, 
                        'leaf_estimation_iterations': 1,
                        'cat_features':cat_features,
                        'one_hot_max_size':4,
                        'eval_metric': 'MultiClass',
                        'class_weights': class_weights,
                        'random_state':42,
                        'task_type':'GPU',
}
    # 파라미터 구성
    cb = CatBoostClassifier(**cbrm_trial_params, verbose = 100)

    # fit the model
    cb.fit(
        train_x.iloc[train_idx], train_y[train_idx],
        eval_set = [(train_x.iloc[valid_idx], train_y[valid_idx])],
        early_stopping_rounds = 50,
        verbose = 100
    )

    # 모델 결과 저장하기
    models.append(cb)
    scores.append(cb.get_best_score()['validation']['MultiClass'])

    # if is_holdout:
    #     break 


0:	learn: 1.3116758	test: 1.3121726	best: 1.3121726 (0)	total: 758ms	remaining: 27m 29s
100:	learn: 0.4066607	test: 0.4320277	best: 0.4320277 (100)	total: 3m 11s	remaining: 1h 5m 30s
200:	learn: 0.3793531	test: 0.4083084	best: 0.4083050 (194)	total: 4m 42s	remaining: 46m 20s
300:	learn: 0.3740661	test: 0.4039020	best: 0.4039015 (297)	total: 5m 44s	remaining: 35m 47s
400:	learn: 0.3553886	test: 0.3897453	best: 0.3897453 (400)	total: 7m 24s	remaining: 32m 47s
500:	learn: 0.3112242	test: 0.3609779	best: 0.3609779 (500)	total: 11m 31s	remaining: 38m 34s
600:	learn: 0.2860304	test: 0.3473459	best: 0.3473459 (600)	total: 15m 33s	remaining: 40m 48s
700:	learn: 0.2666256	test: 0.3373883	best: 0.3373883 (700)	total: 19m 33s	remaining: 41m 12s
800:	learn: 0.2494547	test: 0.3296656	best: 0.3296656 (800)	total: 23m 37s	remaining: 40m 36s
900:	learn: 0.2371519	test: 0.3245392	best: 0.3245392 (900)	total: 27m 25s	remaining: 38m 51s
1000:	learn: 0.2290291	test: 0.3215706	best: 0.3215706 (1000)	total:

In [26]:
# cv 결과 확인
print(scores)
print(np.mean(scores))

[0.304595848955208, 0.29698833998752705, 0.2958271349567797, 0.30075869142626305, 0.30165608564974883, 0.30247034968499303]
0.3003827417767533


In [27]:
# evaluation
def csi_score(CONFUSION_MATRIX, exceptcol = 0):

    # 차원의 수
    n, _ = CONFUSION_MATRIX.shape

    # 계산하여 받을 값
    H = 0
    F = 0
    M = 0
    for i in range(n):
        for j in range(n):
            if i == j == exceptcol:
                continue
            elif i == j:
                H += CONFUSION_MATRIX[i][j]
            elif j != exceptcol:
                F += CONFUSION_MATRIX[i][j]
            elif i != exceptcol:
                M += CONFUSION_MATRIX[i][j]

    return H / (H + F + M)

In [28]:
# csi_score(confusion_matrix(valid_y, cb_pred), 3)

### 5. 추론

- train, valid를 모두 포함한 데이터를 통해 전체적으로 학습을 다시 진행한 다음 test를 적합하도록 한다.  
- 이름을 꼭 ID에 맞도록 설정해 주어야 한다.

In [29]:
# predict
pred_list = []

# 각 모델별 예측값 가져오기
for i, model in enumerate(models):
    pred_list.append(model.predict_proba(test_x))

In [30]:
# 확률값 평균내기
pred_proba = np.mean(pred_list, axis = 0)

In [31]:
# 예측값 가져오기
pred_class = np.argmax(pred_proba, axis = 1) + 1

In [32]:
np.unique(pred_class, return_counts=True)

(array([1, 2, 3, 4], dtype=int64),
 array([   820,   2883,   2328, 256769], dtype=int64))

### 6. 제출물 저장하기

In [33]:
# 제출물 불러오기
submission = pd.read_csv('../Data/fog_test.csv')

In [34]:
# 앞에 필요없는 열 버리기
submission.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [35]:
# 대입해서 저장하기
submission['fog_test.class'] = pred_class

In [36]:
# 저장하기
submission.to_csv('../data/240253.csv', index = False)

### 7. 분석 그 이후

- 생성한 모델에서 어떤 변수가 가장 영향을 주는지 확인할 필요가 있다.

In [37]:
# # # 분석 결과 feature importance 확인
# for model in models:
#     print("=" * 80)
#     explainer = shap.TreeExplainer(model)
#     shap_values = explainer.shap_values(test_x)
    
#     # class_names = [1, 2, 3, 4]
#     # feature importance plot
#     shap.summary_plot(shap_values, train_x, plot_type="bar",
#                   class_inds="original", class_names=model.classes_, feature_names = train_x.columns)
    
#     print("-" * 80)
#     # visualize 
#     shap.summary_plot(shap_values[0], test_x)
#     shap.summary_plot(shap_values[1], test_x)
#     shap.summary_plot(shap_values[2], test_x)
#     shap.summary_plot(shap_values[3], test_x)

In [38]:
# predict
pred_list2 = []

# 각 모델별 예측값 가져오기
for i, model in enumerate(models):
    pred_list2.append(model.predict_proba(train_x))

In [39]:
# 확률값 평균내기
pred_proba2 = np.mean(pred_list2, axis = 0)

In [40]:
# 예측값 가져오기
pred_class2 = np.argmax(pred_proba2, axis = 1) + 1

In [41]:
np.unique(pred_class2, return_counts=True)

(array([1, 2, 3, 4], dtype=int64),
 array([  12683,   25466,   33018, 3085293], dtype=int64))

In [42]:
np.unique(train_y, return_counts=True)

(array([1, 2, 3, 4], dtype=int64),
 array([   7687,   11592,   12207, 3124974], dtype=int64))

In [43]:
pd.crosstab(train_y, pred_class2)

col_0,1,2,3,4
class,,,,
1,6635,832,56,164
2,453,10144,526,469
3,129,1039,9728,1311
4,5466,13451,22708,3083349


In [48]:
# csi score
(6635 + 10144 + 9728)/ (len(train_y) - 3083349)

0.36255830176033704

In [45]:
# pred
train['pred'] = pred_class2

In [46]:
pd.crosstab(train[train['ground'] == 'B']['class'], train[train['ground'] == 'B']['pred'])

pred,1,2,3,4
class,,,,
1,2535,631,30,31
2,215,5942,187,100
3,59,789,3241,313
4,2077,9528,7064,598550


In [47]:
# 1시간 미만만 모아서
fig = px.histogram(train[train['class'] == 2],
                  x = "stn_id",
                  color = 'pred',
                  barmode = "group")

fig.update_layout(title_text="시간별 안개발생 종료 건수 확인",
                    title_x = 0.5,
                    title_xanchor = 'center',
                    title_font_size = 25,
                    title_font_color = 'black',
                    title_font_family = 'NanumSquare',
                    plot_bgcolor='#ffffff')

fig.update_traces(# marker_color = 히스토그램 색, 
                    # marker_line_width = 히스토그램 테두리 두깨,                            
                    # marker_line_color = 히스토그램 테두리 색,
                    marker_opacity = 0.4,
                    )

fig.show()